In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [14]:
class selfattention(nn.Module):
    
    def __init__(self, d_model =2 , row_dim = 0, col_dim = 1):
        super().__init__()
        self.W_q = nn.Linear(in_features=d_model, out_features=d_model, bias = False)
        self.W_k=  nn.Linear(in_features=d_model, out_features=d_model, bias = False)
        self.W_v=  nn.Linear(in_features=d_model, out_features=d_model, bias = False)
        
        self.row_dim = row_dim
        self.col_dim = col_dim
        
    def forward(self, token_embedding):
        x = token_embedding
        q = self.W_q(x)
        k = self.W_k(x)
        v = self.W_v(x)
        
        #(q*k^T)
        
        sims = torch.matmul(q,k.transpose(dim0 = self.row_dim,dim1 =  self.col_dim))
        scaled_sims = sims / torch.tensor(k.size(self.col_dim)**0.5)
        
        attention_percents = F.softmax(scaled_sims, dim = self.col_dim)
        
        attention_score = torch.matmul(attention_percents, V)
        
        return attention_score
        
        
        